Rotator Streanous

This notebook was prepared to do the testing of the rotator in coordination with the pointing kernel and the camera cable wrap.
The goal is to send command to the rotator to go to 90 at azimuth = 0, wait for a minute and then send the rotator to -90 at azimuth = 180

In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [2]:
test_message = "Rotator integration test, Streneous motion"

In [3]:
d = salobj.Domain()

In [4]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [5]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [6]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

[None, None, None]

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 10 messages


In [36]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 70 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 71 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 71 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 72 messages


[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]

Start the loop on the different field positions df given by the scheduler

In [7]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [46]:
print("Azimuth = 0 and rot_tel= 89 deg")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[START]")

# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 60s")
await asyncio.sleep(60.)
    
    
# ########################################################################################### 
    
print("Azimuth = 180 and rot_tel= -89 deg")

alt = 45. * u.deg
az = 180. * u.deg
rot_tel = Angle(-89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[START]")

# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 60s")
await asyncio.sleep(60.)
    
    
# ########################################################################################### 
       
    
#     #we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=30)
await rot.cmd_stop.start(timeout=30)


await asyncio.sleep(5.)
script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"{test_message}:[END]")


RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages


Azimuth = 0 and rot_tel= 89 deg
Time error=-20.08 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 89.0 deg
ParaAng: 179.71047872144564 deg
88d42m37.7234s


RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 23 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 23 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 11 messages


Waiting 60s
Azimuth = 180 and rot_tel= -89 deg
Time error=-0.00 sec
Alt: 45.0 deg
Az: 180.0 deg
RotTel: -89.0 deg
ParaAng: 0.5070657369777498 deg
-268d29m34.5633s
Waiting 60s


True

In [41]:
await rot.cmd_clearError.start(timeout=10)

RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 10 messages


In [42]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 21 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 21 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 21 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 11 messages


[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]